In [10]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

from sklearn.metrics import precision_score, recall_score, accuracy_score, confusion_matrix, f1_score,classification_report

from sklearn.model_selection import GridSearchCV


from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
import joblib

In [2]:
df = pd.read_csv('output.csv')
df

,sex,age,inmsupr,pneumonia,diabetes,asthma,copd,hypertension,cardiovascular,renal_chronic,obesity,tobacco,days_prior_to_treatment,covid_res,intubed,icu,dead
0,0,30,0,1,0,0,0,0,0,0,0,0,86,1,0,0,0
1,1,60,0,0,1,0,0,1,1,0,0,0,0,1,0,0,1
2,0,47,0,1,1,0,0,0,0,0,0,0,0,1,0,1,1
3,0,63,0,0,0,0,0,1,0,0,0,0,9,1,0,0,0
4,1,39,0,1,0,0,0,0,0,0,1,0,5,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87053,0,42,0,0,0,0,0,0,0,0,0,1,6,0,0,0,0
87054,1,1,0,1,0,0,0,0,0,0,0,0,3,0,0,1,0
87055,1,55,0,1,0,0,0,0,0,0,1,0,57,0,0,0,0
87056,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0


In [3]:
df=df.loc[df['covid_res']==1,['sex','age','inmsupr','pneumonia','diabetes','asthma','copd','hypertension','cardiovascular','renal_chronic','obesity','tobacco','days_prior_to_treatment','covid_res','intubed','icu','dead']]
df

,sex,age,inmsupr,pneumonia,diabetes,asthma,copd,hypertension,cardiovascular,renal_chronic,obesity,tobacco,days_prior_to_treatment,covid_res,intubed,icu,dead
0,0,30,0,1,0,0,0,0,0,0,0,0,86,1,0,0,0
1,1,60,0,0,1,0,0,1,1,0,0,0,0,1,0,0,1
2,0,47,0,1,1,0,0,0,0,0,0,0,0,1,0,1,1
3,0,63,0,0,0,0,0,1,0,0,0,0,9,1,0,0,0
4,1,39,0,1,0,0,0,0,0,0,1,0,5,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53550,0,58,0,1,0,0,0,0,0,0,0,0,4,1,1,1,1
53551,0,67,0,1,0,0,0,0,0,0,0,0,115,1,0,0,0
53552,0,65,0,1,1,0,0,1,0,0,1,0,4,1,1,1,0
53553,1,29,0,1,0,0,0,0,0,0,0,0,2,1,0,0,0


In [4]:
X = df.drop(['icu','covid_res','dead','intubed'], axis=1)
y = df['icu']

In [5]:
X

,sex,age,inmsupr,pneumonia,diabetes,asthma,copd,hypertension,cardiovascular,renal_chronic,obesity,tobacco,days_prior_to_treatment
0,0,30,0,1,0,0,0,0,0,0,0,0,86
1,1,60,0,0,1,0,0,1,1,0,0,0,0
2,0,47,0,1,1,0,0,0,0,0,0,0,0
3,0,63,0,0,0,0,0,1,0,0,0,0,9
4,1,39,0,1,0,0,0,0,0,0,1,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
53550,0,58,0,1,0,0,0,0,0,0,0,0,4
53551,0,67,0,1,0,0,0,0,0,0,0,0,115
53552,0,65,0,1,1,0,0,1,0,0,1,0,4
53553,1,29,0,1,0,0,0,0,0,0,0,0,2


In [6]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [8]:
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X, y)
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.30, random_state=0,stratify=y_res)

In [9]:
def reports():
    print("train Accuracy = {}".format(accuracy_score(y_train, train_model.predict(X_train))))
    print("test Accuracy = {}".format(accuracy_score(y_test, test_model)))
    print("Confusion Matrix")
    print(confusion_matrix(y_test, test_model))
    print("Classification Report")
    print(classification_report(y_test, test_model))
    
    

# # random forest 

In [51]:

train_model = RandomForestClassifier()
train_model.fit(X_train, y_train)
test_model = train_model.predict(X_test)

In [52]:
reports()

train Accuracy = 0.9878823017828671
test Accuracy = 0.9130140692640693
Confusion Matrix
[[13600  1184]
 [ 1388 13396]]
Classification Report
              precision    recall  f1-score   support

           0       0.91      0.92      0.91     14784
           1       0.92      0.91      0.91     14784

    accuracy                           0.91     29568
   macro avg       0.91      0.91      0.91     29568
weighted avg       0.91      0.91      0.91     29568



# # rf tuned


In [25]:

train_model = RandomForestClassifier(n_estimators=2200,criterion='entropy',
                             max_features='log2',max_depth=700,min_samples_split=5)
train_model.fit(X_train,y_train)
test_model = train_model.predict(X_test)

In [26]:
reports()

train Accuracy = 0.9836642991737933
test Accuracy = 0.9148403679653679
Confusion Matrix
[[13563  1221]
 [ 1297 13487]]
Classification Report
              precision    recall  f1-score   support

           0       0.91      0.92      0.92     14784
           1       0.92      0.91      0.91     14784

    accuracy                           0.91     29568
   macro avg       0.91      0.91      0.91     29568
weighted avg       0.91      0.91      0.91     29568



# # svm


In [17]:
train_model = SVC()
train_model.fit(X_train, y_train)
test_model = train_model.predict(X_test)

In [18]:
reports()

train Accuracy = 0.7706479199884041
test Accuracy = 0.7704274891774892
Confusion Matrix
[[13753  1031]
 [ 5757  9027]]
Classification Report
              precision    recall  f1-score   support

           0       0.70      0.93      0.80     14784
           1       0.90      0.61      0.73     14784

    accuracy                           0.77     29568
   macro avg       0.80      0.77      0.76     29568
weighted avg       0.80      0.77      0.76     29568



# #svm tuned

In [ ]:
train_model = SVC()
train_model.fit(X_train, y_train)
test_model = train_model.predict(X_test)

In [18]:
reports()

train Accuracy = 0.7706479199884041
test Accuracy = 0.7704274891774892
Confusion Matrix
[[13753  1031]
 [ 5757  9027]]
Classification Report
              precision    recall  f1-score   support

           0       0.70      0.93      0.80     14784
           1       0.90      0.61      0.73     14784

    accuracy                           0.77     29568
   macro avg       0.80      0.77      0.76     29568
weighted avg       0.80      0.77      0.76     29568



# # knn 


In [23]:
train_model = KNeighborsClassifier()
train_model.fit(X_train, y_train)
test_model =train_model.predict(X_test)

In [24]:
reports()

train Accuracy = 0.8882301782867082
test Accuracy = 0.8395224567099567
Confusion Matrix
[[11935  2849]
 [ 1896 12888]]
Classification Report
              precision    recall  f1-score   support

           0       0.86      0.81      0.83     14784
           1       0.82      0.87      0.84     14784

    accuracy                           0.84     29568
   macro avg       0.84      0.84      0.84     29568
weighted avg       0.84      0.84      0.84     29568



# #knn tuned

In [21]:
train_model = KNeighborsClassifier(n_neighbors=5,weights='distance',p=1,metric='minkowski')
train_model.fit(X_train, y_train)
test_model =train_model.predict(X_test)

In [22]:
reports()

train Accuracy = 0.9909407160458037
test Accuracy = 0.8694872835497836
Confusion Matrix
[[12769  2015]
 [ 1844 12940]]
Classification Report
              precision    recall  f1-score   support

           0       0.87      0.86      0.87     14784
           1       0.87      0.88      0.87     14784

    accuracy                           0.87     29568
   macro avg       0.87      0.87      0.87     29568
weighted avg       0.87      0.87      0.87     29568



## gradient boosted trees

In [53]:
train_model = GradientBoostingClassifier()
train_model.fit(X_train, y_train)
test_model =train_model.predict(X_test)

In [54]:
reports()

train Accuracy = 0.8727786635744311
test Accuracy = 0.876758658008658
Confusion Matrix
[[13837   947]
 [ 2697 12087]]
Classification Report
              precision    recall  f1-score   support

           0       0.84      0.94      0.88     14784
           1       0.93      0.82      0.87     14784

    accuracy                           0.88     29568
   macro avg       0.88      0.88      0.88     29568
weighted avg       0.88      0.88      0.88     29568



# #gbt tuned

In [20]:
train_model = GradientBoostingClassifier(max_features='auto', loss='deviance',learning_rate=0.3, max_depth=8,min_samples_leaf=3,min_samples_split=0.1, n_estimators=400, subsample=0.4)
train_model.fit(X_train, y_train)
test_model =train_model.predict(X_test)


In [21]:
reports()

train Accuracy = 0.9354109291201623
test Accuracy = 0.9323593073593074
Confusion Matrix
[[14692    92]
 [ 1908 12876]]
Classification Report
              precision    recall  f1-score   support

           0       0.89      0.99      0.94     14784
           1       0.99      0.87      0.93     14784

    accuracy                           0.93     29568
   macro avg       0.94      0.93      0.93     29568
weighted avg       0.94      0.93      0.93     29568



# # adaboost

In [57]:
train_model = AdaBoostClassifier()
train_model.fit(X_train, y_train)
test_model =train_model.predict(X_test)

In [58]:
reports()

train Accuracy = 0.8104652848238875
test Accuracy = 0.8132778679653679
Confusion Matrix
[[13905   879]
 [ 4642 10142]]
Classification Report
              precision    recall  f1-score   support

           0       0.75      0.94      0.83     14784
           1       0.92      0.69      0.79     14784

    accuracy                           0.81     29568
   macro avg       0.83      0.81      0.81     29568
weighted avg       0.83      0.81      0.81     29568



# #adaboosted tuned

In [59]:
train_model = AdaBoostClassifier(n_estimators= 9000)
train_model.fit(X_train, y_train)
test_model =train_model.predict(X_test)

In [60]:
reports()

train Accuracy = 0.9403681692998985
test Accuracy = 0.9411525974025974
Confusion Matrix
[[14110   674]
 [ 1066 13718]]
Classification Report
              precision    recall  f1-score   support

           0       0.93      0.95      0.94     14784
           1       0.95      0.93      0.94     14784

    accuracy                           0.94     29568
   macro avg       0.94      0.94      0.94     29568
weighted avg       0.94      0.94      0.94     29568



# # xgboost

In [10]:
train_model = XGBClassifier()
train_model.fit(X_train, y_train)
test_model =train_model.predict(X_test)

[18:51:35] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [11]:
reports()

train Accuracy = 0.9328888244673141
test Accuracy = 0.9267789502164502
Confusion Matrix
[[14583   201]
 [ 1964 12820]]
Classification Report
              precision    recall  f1-score   support

           0       0.88      0.99      0.93     14784
           1       0.98      0.87      0.92     14784

    accuracy                           0.93     29568
   macro avg       0.93      0.93      0.93     29568
weighted avg       0.93      0.93      0.93     29568



# #xgboost tuned

In [22]:
train_model = XGBClassifier(colsample_bytree=0.3,learning_rate=0.1,n_estimators=450,objective ='binary:logistic',verbose=1)
train_model.fit(X_train, y_train)
test_model =train_model.predict(X_test)

[05:20:57] WARNING: ..\src\learner.cc:541: 
Parameters: { verbose } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[05:20:58] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [23]:
reports()

train Accuracy = 0.9213219307145963
test Accuracy = 0.9193046536796536
Confusion Matrix
[[14537   247]
 [ 2139 12645]]
Classification Report
              precision    recall  f1-score   support

           0       0.87      0.98      0.92     14784
           1       0.98      0.86      0.91     14784

    accuracy                           0.92     29568
   macro avg       0.93      0.92      0.92     29568
weighted avg       0.93      0.92      0.92     29568

